Заменят ли меня роботы?

Цель:
В этом домашнем задании вам предстоит поработать с методами AutoML и узнать, насколько скоро машины заменят нас всех.

Выберите одно из предложенных соревнований (если хочется - можно найти своё), и попробуйте воспользоваться методами для автоматической генерации признаков и построения моделей, чтобы получить решение. Сравните качество полученных решений с бейзлайном и любой парой моделей "из коробки"

In [23]:
import autofeat
import featuretools as ft
import h2o
from h2o.automl import H2OAutoML
import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [6]:
data_dir = '../data/mercedes-benz-greener-manufacturing'

In [10]:
df = pd.read_csv(f'{data_dir}/train.csv', index_col='ID')

In [8]:
train_df.head()

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
ID,,,,,,,,,,,,,,,,,,,,,
0,130.81,k,v,at,a,d,u,j,o,0,...,0,0,1,0,0,0,0,0,0,0
6,88.53,k,t,av,e,d,y,l,o,0,...,1,0,0,0,0,0,0,0,0,0
7,76.26,az,w,n,c,d,x,j,x,0,...,0,0,0,0,0,0,1,0,0,0
9,80.62,az,t,n,f,d,x,l,e,0,...,0,0,0,0,0,0,0,0,0,0
13,78.02,az,v,n,f,d,h,d,n,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
feature_cols = [col for col in train_df.columns if col.startswith('X')]
binary_feature_cols = [col for col in train_df.columns if col.startswith('X') and int(col[1:]) >= 10]
category_feature_cols = [col for col in train_df.columns if col.startswith('X') and int(col[1:]) < 10]
target_col = 'y'

In [13]:
train_df, valid_df = train_test_split(df, test_size = 0.25, random_state=15)

In [14]:
# Словарь для сохранения рез-тов
r2_dict = dict()

## H2O

### Посмотрим, что нам даст модель подобранная библиотекой h2o

In [15]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_291"; Java(TM) SE Runtime Environment (build 1.8.0_291-b10); Java HotSpot(TM) 64-Bit Server VM (build 25.291-b10, mixed mode)
  Starting server from /Users/valeriamozarova/opt/anaconda3/envs/ml1/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/jn/qpx6dt7s7qg1xsw24ryd1tj40000gn/T/tmp4yk1wzfq
  JVM stdout: /var/folders/jn/qpx6dt7s7qg1xsw24ryd1tj40000gn/T/tmp4yk1wzfq/h2o_valeriamozarova_started_from_python.out
  JVM stderr: /var/folders/jn/qpx6dt7s7qg1xsw24ryd1tj40000gn/T/tmp4yk1wzfq/h2o_valeriamozarova_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Moscow
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.1
H2O_cluster_version_age:,1 month and 6 days
H2O_cluster_name:,H2O_from_python_valeriamozarova_awtdtd
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.778 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [16]:
train_frame = h2o.H2OFrame(train_df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [17]:
train_frame[feature_cols] = train_frame[feature_cols].asfactor()
# for col in binary_feature_cols:
#     train_frame[col] = train_frame[col].asfactor()

In [18]:
aml = H2OAutoML(max_models=20, seed=1, stopping_metric='mse')
aml.train(x=feature_cols, y=target_col, training_frame=train_frame)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)

AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20210501_203540,76.5846,8.75126,76.5846,5.50717,0.0766566
StackedEnsemble_BestOfFamily_AutoML_20210501_203540,76.7241,8.75923,76.7241,5.51616,0.076739
GLM_1_AutoML_20210501_203540,77.3054,8.79235,77.3054,5.56455,0.0771377
XRT_1_AutoML_20210501_203540,77.8584,8.82374,77.8584,5.5811,0.0774824
DeepLearning_1_AutoML_20210501_203540,77.901,8.82615,77.901,5.46484,0.0772535
GBM_5_AutoML_20210501_203540,77.9137,8.82687,77.9137,5.6069,0.0775951
GBM_grid__1_AutoML_20210501_203540_model_2,78.369,8.85263,78.369,5.62916,0.0779205
XGBoost_grid__1_AutoML_20210501_203540_model_3,78.8693,8.88084,78.8693,5.55986,0.0780697
DeepLearning_grid__1_AutoML_20210501_203540_model_1,79.3474,8.90772,79.3474,5.46238,0.0783708
GBM_grid__1_AutoML_20210501_203540_model_1,80.3968,8.96643,80.3968,5.68315,0.0789366


In [19]:
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20210501_203540

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 64.12074676920966
RMSE: 8.007543116912307
MAE: 5.034820697270221
RMSLE: 0.06950767920431186
R^2: 0.6060015524768294
Mean Residual Deviance: 64.12074676920966
Null degrees of freedom: 3155
Residual degrees of freedom: 3146
Null deviance: 513618.9700130638
Residual deviance: 202365.0768036257
AIC: 22109.723747864453

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 76.58463465970488
RMSE: 8.751264746292668
MAE: 5.507170906275232
RMSLE: 0.07665664844358581
R^2: 0.5294155373974871
Mean Residual Deviance: 76.58463465970488
Null degrees of freedom: 3155
Residual degrees of freedom: 3145
Null deviance: 514375.8921559196
Residual deviance: 241701.1069860286
AIC: 22672.319264229824


In [27]:
valid_frame = h2o.H2OFrame(valid_df)
prediction = aml.predict(valid_frame)

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


In [37]:
r2_dict['h2o'] = r2_score(valid_df[target_col].values, h2o.as_list(prediction).values)
print(r2_dict['h2o'])

0.6190350243752996


## Sklearn модель

### Теперь посмотрим рез-тат  базовой модели из коробки

In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split

In [71]:
transformer_steps = [
    ('ohe', OneHotEncoder(handle_unknown='ignore'), category_feature_cols),
]

transformer = ColumnTransformer(transformer_steps, remainder='passthrough')
pipeline = Pipeline([
    ('transform', transformer),
    ('clf', GradientBoostingRegressor())
])

In [72]:
X_train = train_df[feature_cols]
y_train = train_df[target_col].values
X_valid = valid_df[feature_cols]
y_valid = valid_df[target_col].values

In [73]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('transform',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['X0', 'X1', 'X2', 'X3', 'X4',
                                                   'X5', 'X6', 'X8'])])),
                ('clf', GradientBoostingRegressor())])

In [74]:
prediction = pipeline.predict(X_valid)

In [75]:
r2_dict['base_model'] = r2_score(y_valid, prediction)
print(r2_dict['base_model'])

0.6055724119205199


## Попробуем сократить количество фич с помощью PCA

In [76]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from h2o.sklearn import H2OAutoMLRegressor
from h2o.sklearn import H2OGradientBoostingRegressor

pipeline = Pipeline([
    ('transform', transformer),
    ("pca", PCA(n_components=200, random_state=2021)),
    ('clf', GradientBoostingRegressor())
])

In [77]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('transform',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['X0', 'X1', 'X2', 'X3', 'X4',
                                                   'X5', 'X6', 'X8'])])),
                ('pca', PCA(n_components=200, random_state=2021)),
                ('clf', GradientBoostingRegressor())])

In [78]:
prediction = pipeline.predict(X_valid)

In [79]:
r2_dict['pca_model'] = r2_score(y_valid, prediction)
print(r2_dict['pca_model'])

0.5438363146541614


## Boruta. Теперь попробуем отобрать фичи на основе этой библиотеки

In [70]:
from boruta import BorutaPy

In [80]:
transformer_steps = [
    ('ohe', OneHotEncoder(handle_unknown='ignore'), category_feature_cols),
]

transformer = ColumnTransformer(transformer_steps, remainder='passthrough')
clf = GradientBoostingRegressor()
pipeline = Pipeline([
    ('transform', transformer),
    ('feature_select', BorutaPy(clf, n_estimators='auto', verbose=2, random_state=1)),
    ('clf', clf)
])

In [81]:
pipeline.fit(X_train, y_train)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	555
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	555
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	555
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	555
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	555
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	555
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	555
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	10
Rejected: 	545
Iteration: 	9 / 100
Confirmed: 	2
Tentative: 	8
Rejected: 	545
Iteration: 	10 / 100
Confirmed: 	2
Tentative: 	8
Rejected: 	545
Iteration: 	11 / 100
Confirmed: 	2
Tentative: 	8
Rejected: 	545
Iteration: 	12 / 100
Confirmed: 	2
Tentative: 	8
Rejected: 	545
Iteration: 	13 / 100
Confirmed: 	2
Tentative: 	8
Rejected: 	545
Iteration: 	14 / 100
Confirmed: 	2
Tentative: 	8
Rejected: 	545
Iteration: 	15 / 100
Confirmed: 	2
Tentative: 	8
Rejected: 	545
Iteration: 	16 / 100
Confirmed: 	3
Tentative: 	7

Pipeline(steps=[('transform',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['X0', 'X1', 'X2', 'X3', 'X4',
                                                   'X5', 'X6', 'X8'])])),
                ('feature_select',
                 BorutaPy(estimator=GradientBoostingRegressor(n_estimators=149,
                                                              random_state=RandomState(MT19937) at 0x7F9ADC7D8E40),
                          n_estimators='auto',
                          random_state=RandomState(MT19937) at 0x7F9ADC7D8E40,
                          verbose=2)),
                ('clf',
                 GradientBoostingRegressor(n_estimators=149,
                                           random_state=RandomState(MT19937) at 0x7F9ADC7D8E40))])

In [86]:
prediction = pipeline.predict(X_valid)

In [87]:
r2_dict['boruta_base_model'] = r2_score(y_valid, prediction)
print(r2_dict['boruta_base_model'])

0.6127185295118048


## Boruta + H2O

In [114]:
from h2o.sklearn import H2OAutoMLRegressor
from sklearn.preprocessing import FunctionTransformer

In [115]:
transformer_steps = [
    ('ohe', OneHotEncoder(handle_unknown='ignore'), category_feature_cols),
]

transformer = ColumnTransformer(transformer_steps, remainder='passthrough')
clf = GradientBoostingRegressor()
pipeline = Pipeline([
    ('transform', transformer),
    ('feature_select', BorutaPy(clf, n_estimators='auto', verbose=2, random_state=1)),
    ('cls', H2OAutoMLRegressor(max_models=20, seed=1))
])

In [116]:
pipeline.fit(X_train, y_train)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	555
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	555
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	555
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	555
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	555
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	555
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	555
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	10
Rejected: 	545
Iteration: 	9 / 100
Confirmed: 	2
Tentative: 	8
Rejected: 	545
Iteration: 	10 / 100
Confirmed: 	2
Tentative: 	8
Rejected: 	545
Iteration: 	11 / 100
Confirmed: 	2
Tentative: 	8
Rejected: 	545
Iteration: 	12 / 100
Confirmed: 	2
Tentative: 	8
Rejected: 	545
Iteration: 	13 / 100
Confirmed: 	2
Tentative: 	8
Rejected: 	545
Iteration: 	14 / 100
Confirmed: 	2
Tentative: 	8
Rejected: 	545
Iteration: 	15 / 100
Confirmed: 	2
Tentative: 	8
Rejected: 	545
Iteration: 	16 / 100
Confirmed: 	3
Tentative: 	7

Pipeline(steps=[('transform',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['X0', 'X1', 'X2', 'X3', 'X4',
                                                   'X5', 'X6', 'X8'])])),
                ('feature_select',
                 BorutaPy(estimator=GradientBoostingRegressor(n_estimators=149,
                                                              random_state=RandomState(MT19937) at 0x7F9ADD8D1C40),
                          n_estimators='auto',
                          random_state=RandomState(MT19937) at 0x7F9ADD8D1C40,
                          verbose=2)),
                ('cls', H2OAutoMLRegressor(max_models=20, seed=1))])

In [117]:
prediction = pipeline.predict(X_valid)

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


In [118]:
r2_dict['boruta_h2o_model'] = r2_score(y_valid, prediction)
print(r2_dict['boruta_h2o_model'])

0.6110700389141525


In [119]:
r2_dict

{'h2o': 0.6190350243752996,
 'base_model': 0.6055724119205199,
 'pca_model': 0.5438363146541614,
 'boruta_base_model': 0.6127185295118048,
 'boruta_h2o_model': 0.6110700389141525}

Лучше всего показала себя модель H2O без предварительной обработки признаков